# Assessment Value Time Slider Choropleth for Downtown Mall

In [1]:
import pandas as pd
import requests
from folium import plugins

In [2]:
#Import .xls retrieved from the GIS Viewer into pandas

#Path to the .xls retrieve from the GIS Viewer
f = r'./data/mall_parcels_no_ends.xls'
#Create a dataframe that reads the .xls file
df = pd.read_excel(f)
#Identify all rows in df where MULTIPIN column is not equal to 1
not_multipin = df['MULTIPIN'] != 1
#Create a new dataframe that only contains the rows identified in not_multipin
df = df[not_multipin]

## Create Query URL's for open portal data based on .xls data

formatted_gpins = [str(x) for x in df['GPIN'].unique()]
formatted_gpins = formatted_gpins
formatted_gpins = ','.join(formatted_gpins)

# Url for Parcel Area Boundary geojson data
parcel_url = f'https://opendata.arcgis.com/datasets/320d465ddf0d498796da6491e21f6dde_43.geojson?where=GPIN%20in%20({formatted_gpins})'

formatted_pins = [f'%27{x}%27' for x in df['PIN'].unique()]
formatted_pins = formatted_pins
formatted_pins = ','.join(formatted_pins)

assessment_url = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear&outSR=4326&f=json"
assessment_request = requests.get(assessment_url)
assessment_json = assessment_request.json()

# Create data series based on features and combine data frames into a single df
assessment_df = pd.DataFrame(assessment_json['features'])
# Create a single data frame based on combined series data
assessments = pd.DataFrame([x for x in assessment_df['attributes']], dtype = 'object')
assessments = assessments.astype({'TaxYear': 'int64', 'ImprovementValue': 'int64','LandValue': 'int64','TotalValue': 'int64', 'ParcelNumber': 'str'})
assessments = assessments.drop(['ImprovementValue', 'LandValue'], axis=1)
#----------------
#Create data frame from df that holds PIN and GPIN.
df_key = pd.DataFrame(df[['PIN','GPIN']], dtype = 'str')
#----------------
# Merge df_key with assessments
d = pd.merge(assessments, df_key, how='inner', left_on=['ParcelNumber'], right_on=['PIN'])

In [3]:
bins = [0, 100000, 250000, 500000, 1000000, 2000000, 5000000, 10000000, 25000000, 50000000]
bin_color = ['#f7fcfd', '#EEF9FB', '#e5f5f9','#ccece6','#99d8c9','#66c2a4','#41ae76','#238b45','#006d2c','#00441b']

t = d.drop(['ParcelNumber','PIN'], axis = 1)
t.head()
lol = t.values.tolist()
lol = [[x[2],f'{x[0]}-1-1', x[1]] for x in lol]
lol
for row in lol:
    if row[2] in range(bins[0],bins[1]+1):
        row[2] = bin_color[0]
    elif row[2] in range(bins[1],bins[2]+1):
        row[2] = bin_color[1]
    elif row[2] in range(bins[2],bins[3]+1):
        row[2] = bin_color[2]
    elif row[2] in range(bins[3],bins[4]+1):
        row[2] = bin_color[3]     
    elif row[2] in range(bins[4],bins[5]+1):
        row[2] = bin_color[4]
    elif row[2] in range(bins[5],bins[6]+1):
        row[2] = bin_color[5]
    elif row[2] in range(bins[6],bins[7]+1):
        row[2] = bin_color[6]
    elif row[2] in range(bins[7],bins[8]+1):
        row[2] = bin_color[7]
    elif row[2] in range(bins[8],bins[9]+1):
        row[2] = bin_color[8]
    elif row[2] in range(bins[9],bins[10]+1):
        row[2] = bin_color[9]

In [4]:
styledict = {
    'GPIN1': {'2017-1-1': {'color': 'ffffff'},'2017-1-2': {'color': 'fffff0'}},

    'GPIN2': {'2017-1-1': {'color': 'ffffff'},'2017-1-2': {'color': 'fffff0'}}}

In [5]:
df_lol = pd.DataFrame(lol, columns = ['GPIN', 'Year', 'Color'])
df_lol.head()

,GPIN,Year,Color
0,6861,2004-1-1,#66c2a4
1,6861,2003-1-1,#66c2a4
2,6861,2002-1-1,#66c2a4
3,6861,2001-1-1,#66c2a4
4,6861,2000-1-1,#66c2a4


In [7]:
styledata = {}

time_index = df_lol['Year'].unique()
print(time_index)
for gpin in df['GPIN'].unique():
    dfx = pd.DataFrame(df_lol[df_lol['GPIN'] == str(gpin)])
    dfx = pd.DataFrame({'color': dfx['Color']} ,index=time_index)
    print(dfx)
    styledata[gpin] = dfx    
styledata


['2004-1-1' '2003-1-1' '2002-1-1' '2001-1-1' '2000-1-1' '1999-1-1'
 '1998-1-1' '1997-1-1' '2019-1-1' '2018-1-1' '2017-1-1' '2016-1-1'
 '2015-1-1' '2014-1-1' '2013-1-1' '2012-1-1' '2011-1-1' '2010-1-1'
 '2009-1-1' '2008-1-1' '2007-1-1' '2006-1-1' '2005-1-1']
         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1   NaN
2009-1-1   NaN
2008-1-1   NaN
2007-1-1   NaN
2006-1-1   NaN
2005-1-1   NaN
         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1   NaN
2009-1-1   NaN
2008-1-1   NaN
2007-1-1   NaN
2006-1-1   NaN
2005-1-1   NaN
         color
2004-1-

         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1   NaN
2009-1-1   NaN
2008-1-1   NaN
2007-1-1   NaN
2006-1-1   NaN
2005-1-1   NaN
         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1   NaN
2009-1-1   NaN
2008-1-1   NaN
2007-1-1   NaN
2006-1-1   NaN
2005-1-1   NaN
         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1  

         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1   NaN
2009-1-1   NaN
2008-1-1   NaN
2007-1-1   NaN
2006-1-1   NaN
2005-1-1   NaN
         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1   NaN
2009-1-1   NaN
2008-1-1   NaN
2007-1-1   NaN
2006-1-1   NaN
2005-1-1   NaN
         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1  

         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1   NaN
2009-1-1   NaN
2008-1-1   NaN
2007-1-1   NaN
2006-1-1   NaN
2005-1-1   NaN
         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1   NaN
2009-1-1   NaN
2008-1-1   NaN
2007-1-1   NaN
2006-1-1   NaN
2005-1-1   NaN
         color
2004-1-1   NaN
2003-1-1   NaN
2002-1-1   NaN
2001-1-1   NaN
2000-1-1   NaN
1999-1-1   NaN
1998-1-1   NaN
1997-1-1   NaN
2019-1-1   NaN
2018-1-1   NaN
2017-1-1   NaN
2016-1-1   NaN
2015-1-1   NaN
2014-1-1   NaN
2013-1-1   NaN
2012-1-1   NaN
2011-1-1   NaN
2010-1-1  

{7309:          color
 2004-1-1   NaN
 2003-1-1   NaN
 2002-1-1   NaN
 2001-1-1   NaN
 2000-1-1   NaN
 1999-1-1   NaN
 1998-1-1   NaN
 1997-1-1   NaN
 2019-1-1   NaN
 2018-1-1   NaN
 2017-1-1   NaN
 2016-1-1   NaN
 2015-1-1   NaN
 2014-1-1   NaN
 2013-1-1   NaN
 2012-1-1   NaN
 2011-1-1   NaN
 2010-1-1   NaN
 2009-1-1   NaN
 2008-1-1   NaN
 2007-1-1   NaN
 2006-1-1   NaN
 2005-1-1   NaN, 14744:          color
 2004-1-1   NaN
 2003-1-1   NaN
 2002-1-1   NaN
 2001-1-1   NaN
 2000-1-1   NaN
 1999-1-1   NaN
 1998-1-1   NaN
 1997-1-1   NaN
 2019-1-1   NaN
 2018-1-1   NaN
 2017-1-1   NaN
 2016-1-1   NaN
 2015-1-1   NaN
 2014-1-1   NaN
 2013-1-1   NaN
 2012-1-1   NaN
 2011-1-1   NaN
 2010-1-1   NaN
 2009-1-1   NaN
 2008-1-1   NaN
 2007-1-1   NaN
 2006-1-1   NaN
 2005-1-1   NaN, 6656:          color
 2004-1-1   NaN
 2003-1-1   NaN
 2002-1-1   NaN
 2001-1-1   NaN
 2000-1-1   NaN
 1999-1-1   NaN
 1998-1-1   NaN
 1997-1-1   NaN
 2019-1-1   NaN
 2018-1-1   NaN
 2017-1-1   NaN
 2016-1-1   NaN
 2015